This notebook tests out an attempt at defining some `Parameter`-related classes that can be used to flexibily specify either fixed or fitted values that go into a model.

In [ ]:
from experimental_parameters import *
import numpy as np
from pymc3 import Normal, Uniform, Model
from exoplanet import QuadLimbDark

## What kinds of parameters do we need to be able to handle?

Often we just want to put in some fixed value (planet period) or fix array of values (a pair of limb-darkening coefficients).

In [ ]:
period = Fixed(1.234)
period

In [ ]:
ld = Fixed([0.5, 0.6])
ld

Sometimes we might need values that are fixed, where there's one associated with each wavelength. I put this one in for now, but I'm not using it yet, because it has the slightly tricky feature that you would need to know exactly how many wavelengths you're dealing with before you initialize the values, which might not always be the case. But, for example, this could be used for supplying limb-darkening coefficients if we want to fix them exactly to some theoretical values.

In [ ]:
u1_per_wavelength = WavelikeFixed(np.random.normal(0.3, 0.05, 10))
u1_per_wavelength

Often, we have a parameter that we want to be able to fit for, and it's one parameter that applies to the whole multiwavelength model (stellar mass, impact parameter, eccentricity). For PyMC3 fitting, we can set those up by providing the definition of the distribution that's going to be used, but not actually initializing it yet (since that has to happen inside a PyMC3 model). You should think of these as objects that have everything they need to be able to generate a PyMC3 prior. You can provide any other inputs that are needed as additional keywords.

In [ ]:
stellar_radius = Fitted(Normal, mu=1.0, sigma=0.1, name='stellar_radius')
stellar_radius

In [ ]:
ld = Fitted(QuadLimbDark)
ld

There are some parameters that we want to be able to fit for and we want them to have separate values for each wavelength. We don't necessarily know how many wavelengths we're going to be fitting from the start, but the `WavelikeFitted` object will generate as many different parameters as needed when they are needed. 

In [ ]:
radius_ratio = WavelikeFitted(Normal, mu=0.1, sigma=0.02, name='radius_ratio')
radius_ratio

## You can `.get_prior()` from a `Parameter` 

If it's a fixed parameter, this "prior" is just a value.

In [ ]:
period.get_prior()

If it's a fitted parameter, this prior is a PyMC3 distribution.

In [ ]:
with Model():
    print(stellar_radius.get_prior())

If it's a wavelike fixed parameter, the "prior" will be a different value for each index you supply (up to the number of wavelengths).

In [ ]:
u1_per_wavelength.get_prior(0)

In [ ]:
u1_per_wavelength.get_prior(3)

If it's a wavelike fitted parameter, the prior will be a different PyMC3 distribution for each index you supply (up to the number of wavelengths).

In [ ]:
with Model():
    for i in range(10):
        print(radius_ratio.get_prior(i))